# Fit emimTF2N data

EoS: SRK

Parameter T dependence: Quadratic

N (total fitting parameters): 6

**Find an optimal solution, but getting some infeasible points when plotting

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# parmest (binary_param2)
from bip_fitting_functions_SRK import cuadratic

import scipy.stats as stats

## Load Data

In [2]:
data_full = pd.read_csv('../r125_emimtf2n_full.csv')
data_subset = pd.read_csv('../r125_emimtf2n_subset.csv')
data_extended = pd.read_csv('../r125-emimtf2n-extended.csv')

## Load Configuration + Check DOF + Load Parameters

In [3]:
lhs_results1 = pd.read_csv('../Data/Fits/LHS_Fits/SRK_quadTdep_LHS.csv',index_col=0)
lhs_results1 = lhs_results1[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','SSR']]

lhs_results2 = pd.read_csv('Data/Init_Final/LHS_Output/SRK_quadTdep_LHS_final.csv',index_col=0)
lhs_results2 = lhs_results2[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','SSR']]

lhs_results = pd.concat([lhs_results1,lhs_results2],axis=0).reset_index(drop=True)

In [4]:
lhs_results.dropna(axis=0)

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,SSR
11,-0.932453,-0.459271,-0.911759,-0.513797,0.692297,0.936531,11.299369
227,0.140683,-0.506367,-0.463518,0.552634,0.371398,-0.011327,11.299369
710,-1.377136,-1.185180,1.780794,0.699877,-0.476314,0.451712,281.923612
777,-1.416031,-1.550567,-0.251962,-0.119530,1.175178,1.663826,1759.121916
1041,-1.773214,-1.330504,-0.322407,-0.503838,0.674388,1.898514,11.299369
1105,0.048860,-0.351365,-0.906447,0.023691,-1.800902,0.219888,11.299369
1656,-0.292752,-1.431172,-1.593716,1.786987,1.417915,-0.408084,11.299369
1712,-0.047342,0.124671,-1.793649,-1.717538,1.680206,1.340128,4254.684847
1751,-0.496796,0.589789,0.292200,-1.622518,-0.059634,1.001379,11.299368
1809,0.566993,-0.951085,-1.315314,-0.611164,-0.990208,1.563275,11.299369


In [5]:
lhs_best = lhs_results[lhs_results.SSR == lhs_results.SSR.min()]
lhs_best

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,SSR
3983,0.711122,0.175527,-0.938228,-1.881777,-1.441844,1.679764,11.299368


In [6]:
sc_param1 = lhs_best.at[3983, 'sc_param1']
sc_param2 = lhs_best.at[3983, 'sc_param2']
sc_param3 = lhs_best.at[3983, 'sc_param3']
sc_param4 = lhs_best.at[3983, 'sc_param4']
sc_param5 = lhs_best.at[3983, 'sc_param5']
sc_param6 = lhs_best.at[3983, 'sc_param6']

In [7]:
from hfc125_emimtf2n_SRK import configuration 

In [8]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


## Fit Parameters

In [9]:
parameters, obj_value, a = cuadratic(data_subset, configuration, 'R125', 'emimTf2N', "x_R125", "x_emimTf2N", 
        init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
        init_kappa_2_1A = sc_param1, init_kappa_1_2A = sc_param2, 
        init_kappa_2_1B = sc_param3, init_kappa_1_2B = sc_param4,
        init_kappa_2_1C = sc_param5, init_kappa_1_2C = sc_param6,
        eps = 0.1, scaling_fac = 1e-7, 
        filename='Data/Fits/Ipopt_Output/SRK_quadTdep_final.txt')

2022-10-14 12:30:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:30:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:30:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:30:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:30:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:30:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:30:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:30:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:30:38 [INFO] idaes

/Users/bridgettebefort/opt/anaconda3/envs/idaes-pse-dev/lib/python3.9/site-packages/pyomo/contrib/interior_point/interface.py:418: RuntimeWarning: divide by zero encountered in true_divide
  data = (duals_primals_lb/(primals - self._nlp.primals_lb()) +


In [10]:
print('Objective (SSE):',obj_value)

Objective (SSE): 11.299367558621233


In [11]:
print('Parameters:',parameters)

Parameters: fs.properties.SRK_kappa_A[emimTf2N,R125]    0.625729
fs.properties.SRK_kappa_A[R125,emimTf2N]    0.141857
fs.properties.SRK_kappa_B[emimTf2N,R125]   -1.196908
fs.properties.SRK_kappa_B[R125,emimTf2N]   -0.334429
fs.properties.SRK_kappa_C[emimTf2N,R125]    0.501689
fs.properties.SRK_kappa_C[R125,emimTf2N]    0.161541
dtype: float64


In [12]:
params2save = np.zeros(8)
params2save[0] = parameters[1]
params2save[1] = parameters[0]
params2save[2] = parameters[3]
params2save[3] = parameters[2]
params2save[4] = parameters[5]
params2save[5] = parameters[4]

In [13]:
np.savetxt('Data/Fits/SRK_params_quadTdep.csv',params2save,delimiter=',')

In [14]:
cov = np.array(a)
print('Covariance Matrix:')
print(cov)

Covariance Matrix:
[[ 7.39953842e+00  2.41251156e-01 -1.47469630e+01 -4.79067924e-01
   7.32999563e+00  2.37054885e-01]
 [ 2.41251156e-01  3.03379166e-02 -4.55320581e-01 -5.70029438e-02
   2.13282124e-01  2.66168968e-02]
 [-1.47469630e+01 -4.55320581e-01  2.94462441e+01  9.08625670e-01
  -1.46656806e+01 -4.51861066e-01]
 [-4.79067924e-01 -5.70029438e-02  9.08625670e-01  1.07324700e-01
  -4.27982700e-01 -5.02212045e-02]
 [ 7.32999563e+00  2.13282124e-01 -1.46656806e+01 -4.27982700e-01
   7.31969547e+00  2.14034051e-01]
 [ 2.37054885e-01  2.66168968e-02 -4.51861066e-01 -5.02212045e-02
   2.14034051e-01  2.35536924e-02]]


In [15]:
np.savetxt('Data/Covariance/SRK_Cov_quadTdep.csv',cov,delimiter=',')

## Apply Parameters + Plot

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(0.5)

m.fs.properties.SRK_kappa_A['R125','emimTf2N'] = parameters['fs.properties.SRK_kappa_A[R125,emimTf2N]']
m.fs.properties.SRK_kappa_A['emimTf2N','R125'] = parameters['fs.properties.SRK_kappa_A[emimTf2N,R125]']
m.fs.properties.SRK_kappa_B['R125','emimTf2N'] = parameters['fs.properties.SRK_kappa_B[R125,emimTf2N]']
m.fs.properties.SRK_kappa_B['emimTf2N','R125'] = parameters['fs.properties.SRK_kappa_B[emimTf2N,R125]']
m.fs.properties.SRK_kappa_C['R125','emimTf2N'] = parameters['fs.properties.SRK_kappa_C[R125,emimTf2N]']
m.fs.properties.SRK_kappa_C['emimTf2N','R125'] = parameters['fs.properties.SRK_kappa_C[emimTf2N,R125]']

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

R125_x = np.zeros((len(data_extended)))
P = np.zeros((len(data_extended)))
T = np.zeros((len(data_extended)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)

for i in range(len(data_extended)):
    print('i:',i)
    print('Try x_IL:',data_extended["x_emimTf2N"].iloc[i])
    m.fs.liq = data_extended["x_emimTf2N"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data_extended["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(float(data_extended["x_R125"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(data_extended["x_R125"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R125_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R125']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data_extended["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R125_x[i] = np.nan
            P[i] = np.nan
            T[i] = data_extended["temperature"].iloc[i]
            
    except ValueError:
        R125_x[i] = np.nan
        P[i] = np.nan
        T[i] = data_extended["temperature"].iloc[i]
        print('Solver Error')

In [ ]:
fitted_data = pd.DataFrame(data=[R125_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R125_x','P, MPa','T, K']
fitted_data.to_csv('Data/Fits/SRK_FittedData_quadTdep.csv')

In [ ]:
plt.plot(fitted_data.where(fitted_data['T, K']==283.1)['R125_x'],fitted_data.where(fitted_data['T, K']==283.1)['P, MPa'],"b-",label='T=283 K')
plt.plot(fitted_data.where(fitted_data['T, K']==298.1)['R125_x'],fitted_data.where(fitted_data['T, K']==298.1)['P, MPa'],"r-",label='T=298 K')
plt.plot(fitted_data.where(fitted_data['T, K']==323.1)['R125_x'],fitted_data.where(fitted_data['T, K']==323.1)['P, MPa'],"c-",label='T=323 K')
plt.plot(fitted_data.where(fitted_data['T, K']==348.1)['R125_x'],fitted_data.where(fitted_data['T, K']==348.1)['P, MPa'],"m-",label='T=348 K')
plt.plot(data_full["x_R125"],data_full["pressure"]/1e6,"*",color='orange',label="data")  
plt.title('Isotherm R125-[emim][TF2N]')
plt.ylabel('Pressure (MPa)')
plt.xlabel('x R125')
plt.grid(True)
plt.ylim(0,1.110000)
plt.legend()
plt.show()

### Print Eigvals, Eigvectors

An identifiability analysis can be performed with these as well as a FIM because the covariance matrix is the inverse of the FIM. That means that if there are any eigvals > 0, their eigvecs indicate directions of un-identifiability.

In [ ]:
eigvals, eigvecs = np.linalg.eig(cov)

In [ ]:
eigvals

In [ ]:
eigvecs

In [ ]:
np.savetxt('Data/EigenAnalysis/SRK_vals_quadTdep.csv',eigvals,delimiter=',')
np.savetxt('Data/EigenAnalysis/SRK_vecs_quadTdep.csv',eigvecs,delimiter=',')

In [ ]:
FIM = np.linalg.inv(cov) 

#A_opt
FIM_trace = np.trace(FIM)
cov_trace = np.trace(cov)

#D_opt
FIM_det = np.linalg.det(FIM)

#E_opt
eigvals_FIM, eigvecs_FIM = np.linalg.eig(FIM)
eigvals_FIM_min = eigvals_FIM.min()
eigvals_FIM_max = eigvals_FIM.max()

np.savetxt('Data/EigenAnalysis/SRK_FIMtrdet_quadTdep.csv',[cov_trace,FIM_trace,FIM_det,eigvals_FIM_min,eigvals_FIM_max])
np.savetxt('Data/EigenAnalysis/SRK_FIMvals_quadTdep.csv',eigvals_FIM,delimiter=',')
np.savetxt('Data/EigenAnalysis/SRK_FIMvecs_quadTdep.csv',eigvecs_FIM,delimiter=',')

## AIC Value

See: https://doi.org/10.1080/21642583.2018.1496042

AIC(k) = Nln(MSE(k))+2k

N: number of data points

MSE(k): mean squared error of the model with k parameters

k: number of parameters

MSE(k) = SSE(k)/N

SSE: sum of squared error of the model with k parameters

In [ ]:
#number of datapoints
N = len(data_subset)
#MSE(k)
MSE = obj_value*1E-2/N
#number of parameters
k = 6

In [ ]:
AIC = N*np.log(MSE) + 2*k
AIC

In [ ]:
AIC_data = pd.DataFrame(data=[N,MSE,k,obj_value,AIC])
AIC_data = AIC_data.T
AIC_data.columns = ['N','MSE','k','Objective, Pa^2','AIC']
AIC_data.to_csv('Data/AIC/SRK_AIC_quadTdep.csv')